In [1]:
import os
import glob
import pandas as pd

import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool
from scipy import stats
import bokeh_catplot
import pandas as pd
import math

from bokeh.layouts import row
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def bootstrap_sampling(my_array, bootstrap_samples, bootstrap_replicates):
    bs_samples=np.zeros((bootstrap_replicates, bootstrap_samples))
    bs_rep=np.arange(bootstrap_replicates)
    for i in bs_rep:
        bs_samples[i, :]=np.random.choice(my_array, bootstrap_samples)
    return bs_samples

def bootstrap_stats(bs_samples):
    samples_shape=np.shape(bs_samples)
    bs_means=np.zeros((samples_shape[0], 1))
    bs_IC_means=np.zeros((1, 2))
    bs_medians=np.zeros((samples_shape[0], 1))
    bs_IC_medians=np.zeros((1, 2))
    for i in np.arange(samples_shape[0]):
        bs_means[i]=np.mean(bs_samples[i, :])
        bs_medians[i]=np.median(bs_samples[i, :])
    bs_IC_means[0, 0]=np.quantile(bs_means, .025)
    bs_IC_means[0, 1]=np.quantile(bs_means, .975)
    bs_IC_medians[0, 0]=np.quantile(bs_medians, .025)
    bs_IC_medians[0, 1]=np.quantile(bs_medians, .975)
    return np.mean(bs_means), np.median(bs_medians), bs_IC_means, bs_IC_medians

In [4]:
#set root folder
path='D:/Pili_and_PaQa_counts_data/pilB_mNG/solid'
os.chdir(path)
extension = 'csv'
list_csv = []
for root, dirs, files in os.walk(path, topdown=False):
    for name in files:
        if extension in name:
            list_csv.append(os.path.join(root, name))
os.chdir("C:/Users/tala/Desktop/Python-Code/Pole_analysis")
new_dir = "Data\\"
if not os.path.exists(new_dir):
    os.mkdir(new_dir)
os.chdir(new_dir)
combined_csv = pd.concat([pd.read_csv(f) for f in list_csv ], sort=False)
combined_csv.to_csv( "PolesData.csv", index=False, encoding='utf-8-sig')
#combined_csv.head()

In [5]:
df = pd.read_csv('PolesData.csv', sep=',', na_values='*')
df['PercentTotalFluoDim']=df['TotalFluorescencePoleDim']/df['CellTotalFluorescence']
df['PercentTotalFluoBright']=df['TotalFluorescencePoleBright']/df['CellTotalFluorescence']
df['TotalPili']=df['Nb_Pili_PoleDim']+df['Nb_Pili_PoleBright']
df['TotalFlagella']=df['Nb_Flagella_PoleDim']+df['Nb_Flagella_PoleBright']

y_param1='Nb_Pili_PoleDim'
y_param2='Nb_Pili_PoleBright'
tot_pili_limit=4
Ncells_per_pili_Dim=np.empty((tot_pili_limit), dtype='int16')
Ncells_per_pili_Bright=np.empty((tot_pili_limit), dtype='int16')
N=len(df)
print('Dim pole:')
print('NCells='+str(N), end=', ')
for i in range(tot_pili_limit):
    Ncells_per_pili_Dim[i]=len(df.loc[(df[y_param1]==i)])
    if (i<tot_pili_limit-1):
        print('N_'+str(i)+'pili='+str(Ncells_per_pili_Dim[i]), end=', ')
    else: print('N_'+str(i)+'pili='+str(Ncells_per_pili_Dim[i]))
print('Bright pole:')
print('Ncells='+str(N), end=', ')
for i in range(tot_pili_limit):
    Ncells_per_pili_Bright[i]=len(df.loc[(df[y_param2]==i)])
    if (i<tot_pili_limit-1):
        print('N_'+str(i)+'pili='+str(Ncells_per_pili_Bright[i]), end=', ')
    else: print('N_'+str(i)+'pili='+str(Ncells_per_pili_Bright[i]))

Dim pole:
NCells=206, N_0pili=181, N_1pili=19, N_2pili=6, N_3pili=0
Bright pole:
Ncells=206, N_0pili=163, N_1pili=33, N_2pili=7, N_3pili=3


In [6]:
sumPoles=(Ncells_per_pili_Dim+Ncells_per_pili_Bright)
Prob_Dim_if_nb_pili = [(Ncells_per_pili_Dim[i]/sumPoles[i]) for i in range(tot_pili_limit) if sumPoles[i]>0]
Prob_Bright_if_nb_pili=[(Ncells_per_pili_Bright[i]/sumPoles[i]) for i in range(tot_pili_limit) if sumPoles[i]>0]

Prob_per_pili_Dim= Ncells_per_pili_Dim/N
Prob_per_pili_Bright= Ncells_per_pili_Bright/N

In [7]:
p0 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# pili', 
    y_axis_type='linear',
    y_axis_label ='P(# pili)',
    title="Probability of having # pili"
)

x_1=range(tot_pili_limit)


p0.line(
    x=x_1,
    y=Prob_per_pili_Dim, 
    line_color = 'blue',
    #fill_color = 'blue',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Dim Pole'
)


p0.line(
    x=x_1,
    y=Prob_per_pili_Bright, 
    line_color = 'red',
    #fill_color = 'red',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Bright pole'
)


bokeh.io.show(p0)

In [8]:
p1 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# pili', 
    y_axis_type='linear',
    y_axis_label ='P(# pili|pole state)',
    title="Probability of having # pili if bright or dim pole"
)

x_1=range(len(Prob_Dim_if_nb_pili))


p1.line(
    x=x_1,
    y=Prob_Dim_if_nb_pili, 
    line_color = 'blue',
    #fill_color = 'blue',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Dim Pole'
)


p1.line(
    x=x_1,
    y=Prob_Bright_if_nb_pili, 
    line_color = 'red',
    #fill_color = 'red',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Bright pole'
)


bokeh.io.show(p1)

In [9]:
nb_pili=np.zeros((tot_pili_limit, 1))
boot_meanDim=np.zeros((tot_pili_limit, 1))
boot_medianDim=np.zeros((tot_pili_limit, 1))
boot_IC_meanDim=np.zeros((tot_pili_limit, 2))
boot_IC_medianDim=np.zeros((tot_pili_limit, 2))
boot_meanBright=np.zeros((tot_pili_limit, 1))
boot_medianBright=np.zeros((tot_pili_limit, 1))
boot_IC_meanBright=np.zeros((tot_pili_limit, 2))
boot_IC_medianBright=np.zeros((tot_pili_limit, 2))
for n_pili in range(tot_pili_limit):
    cell_fluorescence_arrayDim=df.TotalFluorescencePoleDim.loc[(df[y_param1]==n_pili)].values
    cell_fluorescence_arrayBright=df.TotalFluorescencePoleBright.loc[(df[y_param2]==n_pili)].values
    nb_pili[n_pili]=n_pili
    if (len(cell_fluorescence_arrayDim) > 0):
        bs_Dim_fluorescence_array=bootstrap_sampling(cell_fluorescence_arrayDim, len(cell_fluorescence_arrayDim), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_Dim_fluorescence_array)
        boot_meanDim[n_pili]=bs_means
        boot_IC_meanDim[n_pili,:]=IC_means
        boot_medianDim[n_pili]=bs_medians
        boot_IC_medianDim[n_pili,:]=IC_medians
    if (len(cell_fluorescence_arrayBright) > 0):
        bs_Bright_fluorescence_array=bootstrap_sampling(cell_fluorescence_arrayBright, len(cell_fluorescence_arrayBright), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_Bright_fluorescence_array)
        boot_meanBright[n_pili]=bs_means
        boot_IC_meanBright[n_pili,:]=IC_means
        boot_medianBright[n_pili]=bs_medians
        boot_IC_medianBright[n_pili,:]=IC_medians

names = ['TotalPili', 'bootMeanDim','bootMedianDim', 'bootMeanBright','bootMedianBright']
data = np.concatenate((nb_pili, boot_meanDim, boot_medianDim, boot_meanBright, boot_medianBright), axis=1)
df_boot_stats=pd.DataFrame(data=np.transpose(data), index=names).T
df_boot_stats['IC_meanDim']=list(boot_IC_meanDim)
df_boot_stats['IC_medianDim']=list(boot_IC_medianDim)
df_boot_stats['IC_meanBright']=list(boot_IC_meanBright)
df_boot_stats['IC_medianBright']=list(boot_IC_medianBright)
df_boot_stats

,TotalPili,bootMeanDim,bootMedianDim,bootMeanBright,bootMedianBright,IC_meanDim,IC_medianDim,IC_meanBright,IC_medianBright
0,0.0,8002.040337,7605.0,14361.976252,13396.0,"[7540.33453038674, 8453.007872928178]","[7068.0, 8235.0]","[13428.263343558283, 15425.142024539877]","[12145.0, 14625.0]"
1,1.0,11302.117105,9768.0,18836.649000,18117.0,"[8823.531578947368, 14228.497368421049]","[8363.0, 11397.0]","[16170.533333333333, 21848.67424242424]","[15057.0, 20650.0]"
2,2.0,10671.122167,11396.0,21305.879857,19715.0,"[7940.312500000001, 12775.5]","[7240.5, 13337.5]","[18874.55, 24555.14285714286]","[18856.0, 25323.0]"
3,3.0,0.000000,0.0,24536.629000,15753.0,"[0.0, 0.0]","[0.0, 0.0]","[12441.0, 46086.0]","[12441.0, 46086.0]"


In [10]:
x_param1='TotalFluorescencePoleDim'#'PercentTotalFluoDim'
y_param1='Nb_Pili_PoleDim'
x_param2='TotalFluorescencePoleBright' #'PercentTotalFluoBright'
y_param2='Nb_Pili_PoleBright'
param2='TotalPili'
IC1='IC_medianDim'
IC2='IC_medianBright'
param1='bootMedianDim'
param3='bootMedianBright'
p2 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# Pili', 
    y_axis_type='linear',
    y_axis_label ='Pole Total fluorescence',
    title="Pole fluorescence vs #Pili (bootstrap median, 95% CI)"
)

p2.circle(
    source=df,
    x=y_param1,
    y=x_param1, 
    line_color = 'blue',
    fill_color = 'blue',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Dim Pole'
)
p2.circle(
    source=df,
    x=y_param2,
    y=x_param2, 
    line_color = 'red',
    fill_color = 'red',
    alpha=0.7,
    #legend = labelsAll[i]
    legend = 'Bright Pole'
)

for n_pili in list(df_boot_stats[param2]):
    b = [n_pili,n_pili]
    ic=df_boot_stats.loc[(df_boot_stats[param2] == n_pili), [IC1]].values[0][0]
    p2.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=0.7,
        line_width=3
    )
    
for n_pili in list(df_boot_stats['TotalPili']):
    b = [n_pili,n_pili]
    ic=df_boot_stats.loc[(df_boot_stats[param2] == n_pili), [IC2]].values[0][0]
    p2.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=0.7,
        line_width=3
    )

p2.line(
    source = df_boot_stats.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'blue',
    alpha=0.6
)


p2.circle(
    source = df_boot_stats.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'black',
    fill_color = 'white',
    alpha=0.6,
    size=10
)

p2.line(
    source = df_boot_stats.loc[:, [param2, param3]],
    x = param2,
    y = param3,
    line_color = 'red',
    alpha=0.6
) 

p2.circle(
    source = df_boot_stats.loc[:, [param2, param3]],
    x = param2,
    y = param3,
    line_color = 'black',
    fill_color = 'white',
    alpha=0.6,
    size=10
)  

p2.output_backend = 'svg'

bokeh.io.show(p2)

In [11]:
x_param3='CellTotalFluorescence'
y_param3='PolarRatio'

print(x_param3+' vs '+y_param3+':')
[spearman_r, spearman_p]=stats.spearmanr(df[x_param3], df[y_param3])
print('Spearman correlation = '+str(spearman_r)+', p-value = '+ str(spearman_p))

[pearson_r, pearson_p]=stats.pearsonr(df[x_param3], df[y_param3])
print('Pearson correlation = '+str(pearson_r)+', p-value = '+str(pearson_p))

CellTotalFluorescence vs PolarRatio:
Spearman correlation = 0.5498426491229876, p-value = 1.1327040735719129e-17
Pearson correlation = 0.5159693063705415, p-value = 2.069171785329773e-15


In [12]:
x_param3='CellTotalFluorescence'
y_param3='PolarRatio'

p1 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='Total fluorescence', 
    y_axis_type='linear',
    y_axis_label ='Polar Ratio',
    title="Polar Ratio vs Cell Total fluorescence"
)

p1.circle(
    source=df,
    x=x_param3,
    y=y_param3, 
    line_color = 'black',
    fill_color = 'gray',
    alpha=0.7,
    #legend = labelsAll[i]
)

bokeh.io.show(p1)

In [13]:
y_param1='Nb_Pili_PoleDim'
y_param2='Nb_Pili_PoleBright'
x_param1='TotalFluorescencePoleDim'
x_param2='TotalFluorescencePoleBright'
parameterX='TotalPili'
parameterY='PoleTotalFluorescence'
poles_pili_dim=list(df[y_param1])
poles_pili_bright=list(df[y_param2])
poles_pili=poles_pili_dim+poles_pili_bright
poles_pili=np.asarray(poles_pili).reshape(len(poles_pili),1)
poles_fluo_dim=list(df[x_param1])
poles_fluo_bright=list(df[x_param2])
poles_fluo=poles_fluo_dim+poles_fluo_bright
poles_fluo=np.asarray(poles_fluo).reshape(len(poles_fluo),1)

names = [parameterX, parameterY]
data = np.concatenate((poles_pili, poles_fluo), axis=1)
df_long=pd.DataFrame(data=np.transpose(data), index=names).T

In [14]:
x_param3=parameterX
y_param3=parameterY

print(x_param3+' vs '+y_param3+':')
[spearman_r, spearman_p]=stats.spearmanr(df_long[x_param3], df_long[y_param3])
print('Spearman correlation = '+str(spearman_r)+', p-value = '+ str(spearman_p))

[pearson_r, pearson_p]=stats.pearsonr(df_long[x_param3], df_long[y_param3])
print('Pearson correlation = '+str(pearson_r)+', p-value = '+str(pearson_p))

TotalPili vs PoleTotalFluorescence:
Spearman correlation = 0.27789112472898175, p-value = 9.647240114809636e-09
Pearson correlation = 0.3052332006048625, p-value = 2.477443104386947e-10


In [15]:
nb_pili=np.zeros((tot_pili_limit, 1))
boot_mean=np.zeros((tot_pili_limit, 1))
boot_median=np.zeros((tot_pili_limit, 1))
boot_IC_mean=np.zeros((tot_pili_limit, 2))
boot_IC_median=np.zeros((tot_pili_limit, 2))

for n_pili in range(tot_pili_limit):
    Pole_fluorescence_array=df_long.PoleTotalFluorescence.loc[(df_long[parameterX]==n_pili)].values
    nb_pili[n_pili]=n_pili
    if (len(Pole_fluorescence_array) > 0):
        bs_fluorescence_array=bootstrap_sampling(Pole_fluorescence_array, len(Pole_fluorescence_array), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_fluorescence_array)
        boot_mean[n_pili]=bs_means
        boot_IC_mean[n_pili,:]=IC_means
        boot_median[n_pili]=bs_medians
        boot_IC_median[n_pili,:]=IC_medians

names = ['TotalPili', 'bootMean','bootMedian']
data = np.concatenate((nb_pili, boot_mean, boot_median), axis=1)
df_boot_stats_tot=pd.DataFrame(data=np.transpose(data), index=names).T
df_boot_stats_tot['IC_mean']=list(boot_IC_mean)
df_boot_stats_tot['IC_median']=list(boot_IC_median)
df_boot_stats_tot

,TotalPili,bootMean,bootMedian,IC_mean,IC_median
0,0.0,10986.728480,9540.0,"[10331.063299418605, 11606.721002906976]","[9100.0, 10234.5]"
1,1.0,16128.917885,14853.5,"[13843.986538461539, 18552.920673076922]","[11321.0, 18660.0]"
2,2.0,16328.157538,16955.0,"[12912.426923076922, 19845.015384615388]","[12757.0, 19715.0]"
3,3.0,24620.129000,15753.0,"[12441.0, 46086.0]","[12441.0, 46086.0]"


In [17]:
param2='TotalPili'
IC1='IC_median'
param1='bootMedian'

p3 = bokeh.plotting.figure(
    width=600, 
    height=600, 
    x_axis_label='# pili', 
    y_axis_type='linear',
    y_axis_label ='pole fluorescence',
    title="Pole total fluorescence vs # pili (bootstrap median, 95% CI)"
)

p3.circle(
    source=df_long,
    x=parameterX,
    y=parameterY, 
    line_color = 'green',
    fill_color = 'green',
    alpha=0.4,
    #legend = labelsAll[i]
)

for n_pili in list(df_boot_stats_tot[param2]):
    b = [n_pili,n_pili]
    ic=df_boot_stats_tot.loc[(df_boot_stats_tot[param2] == n_pili), [IC1]].values[0][0]
    p3.line(
        x = b,
        y = ic,
        color = 'black',
        alpha=0.7,
        line_width=3
    )

p3.line(
    source = df_boot_stats_tot.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'black',
    alpha=0.6
)


p3.circle(
    source = df_boot_stats_tot.loc[:, [param2, param1]],
    x = param2,
    y = param1,
    line_color = 'black',
    fill_color = 'white',
    alpha=0.6,
    size=10
)

p3.output_backend = 'svg'

bokeh.io.show(p3)